In [1]:
def parent():
    print("parent")
    def first_child():
        print("first child")

In [2]:
parent()

parent


first child is not a function defined globally. It only exits in the local namespace of parent()

In [3]:
first_child()

NameError: name 'first_child' is not defined

In [4]:
def parent():
    print("parent")
    def first_child():
        print("first child")
    print(locals())

In [6]:
parent()

parent
{'first_child': <function parent.<locals>.first_child at 0x7f69a40e0a60>}



you can also return functions

In [7]:
def parent(x):
    print("parent")
    def first_child():
        print("first child")
    def second_child():
        print("second child")
    if x == 1:
        return first_child
    else:
        return second_child

In [9]:
f = parent(1)

parent


In [11]:
f()

first child


fascinating 

# decorator

In [12]:
def my_decorator(func):
    def wrapper():
        print("wrapper of decorator")
        func()
    return wrapper

In [15]:
my_decorator(f)()

wrapper of decorator
first child


In [22]:
f = my_decorator(f)
#this notation is important to understand more complex decorators

In [17]:
f()

wrapper of decorator
first child


In [18]:
from datetime import datetime

def my_decorator(func):
    def wrapper():
        if datetime.now().hour < 8:
            func()
            
    return wrapper

In [19]:
@my_decorator
def morning():
    print("good morning")

In [21]:
morning()
#it won't print anything because the time is after 8 and @my_decorator is a syntactic sugar for morning = my_decorator(morning)

In [23]:
#another simple decorator
def do_twice(func):
    def wrapper():
        func()
        func()
        
    return wrapper

In [26]:
@do_twice
def morning():
    print("good morning")

In [27]:
morning()

good morning
good morning


tenacity is a library it has decoraor called retry

it will rerun the function

(it has a exception)

when you do @do_twice>>>> you have overridden the function so you don't have the original one anymore

@do_twice : it applies to the function that is defined right after it.

# how to handle inputs?

In [28]:
def do_twice(func):
    def wrapper(name):
        func(name)
        func(name)
        
    return wrapper

In [29]:
def say_hi(name):
    print(f"hi {name}")

In [30]:
say_hi = do_twice(say_hi)

In [31]:
say_hi("jose")

hi jose
hi jose


what happens if we change the number of inputs?
it gets super hard!!!
what to do?

In [32]:
def do_twice(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
        
    return wrapper

In [34]:
@do_twice
def say_hi(name):
    print(f"hi {name}")

In [35]:
say_hi("jose")

hi jose
hi jose


In [36]:
@do_twice
def say_stuff(*args):
    i = 1
    for arg in args:
        print(f'the {i} argument is {arg}')
        i += 1

In [37]:
say_stuff("jose", "is", "cool")

the 1 argument is jose
the 2 argument is is
the 3 argument is cool
the 1 argument is jose
the 2 argument is is
the 3 argument is cool


In [38]:
@do_twice
def banana(i):
    print(f'I have {i} banana(s)')

In [39]:
banana(5)

I have 5 banana(s)
I have 5 banana(s)


what about output?
just make wrapper return one


In [40]:
def do_twice(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        output = func(*args, **kwargs)
        return output
    return wrapper

In [41]:
@do_twice
def three(i):
    return 3 * i

In [42]:
print(three(5))

15


In [43]:
three

<function __main__.do_twice.<locals>.wrapper(*args, **kwargs)>

as you can see its name is wrapper

it has completely changed identity

we lose all documentation!!

In [44]:
@do_twice
def say_hi(name):
    """say hi to someone

    Args:
        name (_type_): _description_
    """
    print(f"hi {name}")

In [45]:
help(say_hi)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



as you can see documentation is lost and its name is wrapper!!

what can we do to not lose this much information?

In [46]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper

#the interesting thing is that to keep the docstring of the function we need to use decorators ( @functools.wraps(func) )

In [47]:
@do_twice
def say_hi(name):
    """say hi to someone

    Args:
        name (_type_): _description_
    """
    print(f"hi {name}")
say_hi("jose")

hi jose
hi jose


In [48]:
say_hi.__name__

'say_hi'

In [49]:
help(say_hi)

Help on function say_hi in module __main__:

say_hi(name)
    say hi to someone
    
    Args:
        name (_type_): _description_



In [50]:
dir(say_hi)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__wrapped__']

another example:

timer decorator


In [64]:
import time

def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        output = func(*args, **kwargs)
        stop = time.time()
        print(f"the function {func.__name__} took {stop - start:0.3f} seconds")
        return output
    
    return wrapper

In [65]:
@timer
def f():
    for i in range(10000):
        q = i * i
    return q - 1

In [66]:
f()

the function f took 0.001 seconds


99980000

In [69]:
def f(r, *args):
    print(args)
    print(type(args))

In [70]:
f(1, 2, 3, 4, 5)

(2, 3, 4, 5)
<class 'tuple'>


In [72]:
#that was just a memory test for myself

In [73]:
def debug(func):
    @functools.wraps(func)
    def wrapper(*args, **kargs):
        print(f'the function name {func.__name__} and the number of arguments {len(args) + len(kargs)}')
        
        output = func(*args, **kargs)
        print(f'the function {func.__name__} returned {output}')
        return output
    
    return wrapper

In [80]:
@debug
def f(x, y, z):
    say_hi("jose")
    return x + y + z

f(1, 2, z = 5)

the function name f and the number of arguments 3
hi jose
hi jose
the function f returned 8


8

say_hi is not debugged obviously

In [81]:
say_hi = debug(say_hi)
f(1, 2, z = 5)

the function name f and the number of arguments 3
the function name say_hi and the number of arguments 1
hi jose
hi jose
the function say_hi returned None
the function f returned 8


8

this can be used to debug functions that we don't have access to their code!

In [79]:
d = {'x': 1, 'y': 2, 'z': "3"}
for key, value in d.items():
    print(f'{key}, {value!r}')

x, 1
y, 2
z, '3'
